# LAD geocoding

Here we geocode GtR organisations at the LAD level

## 0. Preamble

In [ ]:
%run notebook_preamble.ipy

In [ ]:
def flatten_list(a_list):
    return([x for el in a_list for x in el])


### Imports

In [ ]:
#For geocoding into lads
import geopandas as gp

from shapely.geometry import Point

### Lad shapefile

In [ ]:
lad_shape = gp.read_file(
    '../data/external/Local_Authority_Districts_December_2017_Full_Clipped_Boundaries_in_Great_Britain.shp')

### Change the projection

In [ ]:
lad_shape.to_crs(epsg=4326,inplace=True)

### Lookups

In [ ]:
scottish_lads = {'Aberdeen City',
 'Aberdeenshire',
 'Angus',
 'Argyll and Bute',
 'Clackmannanshire',
 'Dumfries and Galloway',
 'Dundee City',
 'East Ayrshire',
 'East Dunbartonshire',
 'East Lothian',
 'East Renfrewshire',
 'Edinburgh, City of',
 'Eilean Siar',
 'Falkirk',
 'Fife',
 'Glasgow City',
 'Highland',
 'Inverclyde',
 'Midlothian',
 'Moray',
 'North Ayrshire',
 'North Lanarkshire',
 'Orkney Islands',
 'Perth and Kinross',
 'Renfrewshire',
 'Scottish Borders',
 'Shetland Islands',
 'South Ayrshire',
 'South Lanarkshire',
 'Stirling',
 'West Dunbartonshire',
 'West Lothian'}

### Some renaming

In [ ]:
rename_lads = {'E07000146': 'King`s Lynn and West Norfolk',
 'E07000112': 'Shepway',
 'W06000001': 'Anglesey',
 'W06000014': 'The Vale of Glamorgan',
 'W06000016': 'Rhondda, Cynon, Taff',
 'S12000036': 'Edinburgh, City of',
 'S12000013': 'Eilean Siar',
 '95AA': 'Antrim',
 '95BB': 'Ards',
 '95CC': 'Armagh',
 '95DD': 'Ballymena',
 '95EE': 'Ballymoney',
 '95FF': 'Banbridge',
 '95GG': 'Belfast',
 '95HH': 'Carrickfergus',
 '95II': 'Castlereagh',
 '95JJ': 'Coleraine',
 '95KK': 'Cookstown',
 '95LL': 'Craigavon',
 '95MM': 'Derry',
 '95NN': 'Down',
 '95OO': 'Dungannon',
 '95PP': 'Fermanagh',
 '95QQ': 'Larne',
 '95RR': 'Limavady',
 '95SS': 'Lisburn',
 '95TT': 'Magherafelt',
 '95UU': 'Moyle',
 '95VV': 'Newry and Mourne',
 '95WW': 'Newtownabbey',
 '95XX': 'North Down',
 '95YY': 'Omagh',
 '95ZZ': 'Strabane'}

In [ ]:
lad_shape['lad_name'] = [rename_lads[x] if x in rename_lads.keys() else name for x,name in
                        zip(lad_shape['lad17cd'],lad_shape['lad17nm'])]

## 1. Load files and spatial join

In [ ]:
gtr_dir = '../data/raw/gtr/2019-05-02/'

orgs, orgs_locs = [pd.read_csv(gtr_dir+name) for name in ['/gtr_organisations.csv','/gtr_organisations_locations.csv']]

In [ ]:
orgs_locs['coordinates'] = orgs_locs[['longitude','latitude']].apply(Point,axis=1)

In [ ]:
org_locs = gp.GeoDataFrame(orgs_locs,geometry='coordinates')

Spatial join (point in polygon)

In [ ]:
lad_gtr = gp.sjoin(org_locs,lad_shape,op='within')

In [ ]:
lad_gtr.lad_name.value_counts().head()

In [ ]:
len(lad_gtr)

In [ ]:
len(org_locs)

In [ ]:
matched_ids = set(lad_gtr['id'])

orgs_locs.loc[[x not in matched_ids for x in org_locs['id']]]['country_name'].value_counts().head()

Most of the unmatched orgs have missing geographical information

In [ ]:
#Create an org id - lad lookup
org_lad_lookup = {x['id']:[x['lad17cd'],x['lad_name']] for n,x in lad_gtr.iterrows()}

## Create dfs for matching

I want a df where every row is a project. The columns represent:

* The LAD of the lead organisation
* The LADs of the participant organisations
* Flags for whether the lead and participating organisations are Scottish or not

In [ ]:
link = pd.read_csv('../data/raw/gtr/2019-06-13/gtr_link_table.csv')

In [ ]:
org_link = link.loc[['_ORG' in x for x in link['rel']]].reset_index(drop=False)

In [ ]:
org_link.columns

In [ ]:
org_link_grouped = org_link.groupby(['project_id','rel'])['id'].apply(lambda x: list(set(x))).reset_index(drop=False)

In [ ]:
org_link_grouped_wide = pd.pivot_table(org_link_grouped,index='project_id',columns='rel',values='id',aggfunc=lambda x: list(x)[0])

#### Run the lookup - we need some nested loops to deal with missing values and missing orgs

In [ ]:
def lad_allocator(var_name,df):
    '''
    Looks up the lad code and name of organisations participating in a project
    '''
    
    df[f'{var_name.lower()}_lad_code'],df[f'{var_name.lower()}_lad_name'] = [
    [[] if type(x)==float else [org_lad_lookup[el][n] for el in x if el in org_lad_lookup.keys()] for x in df[f'{var_name}_ORG']] for
    n in [0,1]]
    
    return(df)
    
    

Each of these returns a geolabelled dataset

In [ ]:
org_geo = lad_allocator('LEAD',org_link_grouped_wide)
org_geo = lad_allocator('PARTICIPANT',org_link_grouped_wide)
org_geo = lad_allocator('PP',org_link_grouped_wide)
org_geo = lad_allocator('COLLAB',org_link_grouped_wide)
org_geo = lad_allocator('FELLOW',org_link_grouped_wide)

We group all the organisation geo data, and all the involved (all except the lead) in two lists

In [ ]:
org_geo['all_lad_code'],org_geo['all_lad_name'] = [[flatten_list([row[name+f'_lad_{var}'] for name in ['lead','participant','pp','collab','fellow']]) for
                                                             n, row in org_geo.iterrows()] for var in ['code','name']]


org_geo['involved_lad_code'],org_geo['involved_lad_name'] = [[flatten_list([row[name+f'_lad_{var}'] for name in ['participant','pp','collab','fellow']]) for
                                                             n, row in org_geo.iterrows()] for var in ['code','name']]

In [ ]:
org_geo.to_csv(f'../data/temp_scotland/{today_str}_gtr_org_lad_labelled.csv',compression='zip')

## Merge with the combined df

In [ ]:
df = pd.read_csv('../data/processed/14_6_2019_combined_gtr_projects.csv',compression='zip')
df = df[[x for x in df.columns if 'Unnamed' not in x]]

In [ ]:
df_w_geo = pd.merge(df,org_geo,left_on='project_id',right_on='project_id')

In [ ]:
len(df)-len(df_w_geo)

In [ ]:
matched_ids = set(df_w_geo.project_id)

unmatched = df.loc[[x not in matched_ids for x in df['project_id']]]

In [ ]:
unmatched_ids = set(unmatched['project_id'])

In [ ]:
link.loc[[x in unmatched_ids for x in link['project_id']]]['rel'].value_counts()

No organisation data for the unmatched ones! 

**Todo** check with Joel and Russ about this

In [ ]:
#df_w_geo_w_lead = df_w_geo.dropna(axis=0,subset=['lead_lad_name'])

In [ ]:
df_w_geo['lead_scot'],df_w_geo['inv_scot'] = [[any(el in scottish_lads for el in x) for x in df_w_geo[var]] for var in ['lead_lad_name','involved_lad_name']]

In [ ]:
df_w_geo['inv_scot_n'] = [np.sum([el in scottish_lads for el in x]) for x in df_w_geo['all_lad_name']]

In [ ]:
df_w_geo.to_csv(f'../data/temp_scotland/{today_str}_gtr_projects_geo_labelled.csv',compression='zip')

In [ ]:
pd.Series(flatten_list(df_w_geo['lead_lad_name'])).value_counts()

## Organisation to project lookup

In [ ]:
#Org focused has two columns, one with lists of projects an organisation has participated in and another with the list of roles 
#org_focused = pd.concat([org_link.groupby('id')[var].apply(lambda x: list(x)) for var in ['project_id','rel']],axis=1)


org_focused = org_link.set_index('id').iloc[:,2:] #The positional indexing is to remove some unnecessary variables

In [ ]:
#Org name lookup
org_name_lookup,org_lad_lookup = [df.set_index('id').to_dict(orient='index') for df in [orgs,lad_gtr]]

In [ ]:
#Label the org_focused dataframe with names and places

org_focused['name'],org_focused['lad'] = [[df[org_id][var] if org_id in df.keys() else np.nan for org_id in org_focused.index] for df,var in zip(
    [org_name_lookup,org_lad_lookup],['name','lad_name'])]

org_focused.reset_index(drop=False,inplace=True)


### Load the enriched project dataset

In [ ]:
from ast import literal_eval

In [ ]:
proj_meta = pd.read_csv('../data/temp_scotland/21_5_2019_gtr_projects_geo_labelled.csv',compression='zip')

In [ ]:
org_proj = pd.merge(org_focused,proj_meta,left_on='project_id',right_on='project_id')

In [ ]:
#We want to parse the lists in the data

list_var = [x for x in org_proj.columns if '_lad_' in x]

#If the column is in the list above, then parse it
for c in org_proj.columns:
    
    if c in list_var:
        org_proj[c] = [literal_eval(x) for x in org_proj[c]]


In [ ]:
org_proj.head()

In [ ]:
#Create a couple of variables that will help with the analysis later

#This is capturing if an organisation is in a local collaboration

org_proj['local_collab'] = [x['all_lad_name'].count(x['lad'])>1 if pd.isnull(x['lad'])==False else np.nan for pid,x in org_proj.iterrows()]

#It will easy to turn this into a measure of Scottish local collaborations (they are local collaborations for organisations in Scotland)

In [ ]:
#And we create another about Scottish collaborations
org_proj['scot_collab'] = [(x['lad'] in scottish_lads) & (len(scottish_lads & set(x['all_lad_name']))>1) for pid, x in org_proj.iterrows()]

Now we want to tidy up this df

What variables do we want to create?

* Total number of projects
* Total number of projects led
* Total level of funding
* Total level of funding in projects led
* Discipline distribution (projects led)
* Funder distribution
* Output distribution (projects involving)
* Local collaborations
* Scottish local collaborations
* Scottish local collaborations
* Top Scottish collaborator (?)

In [ ]:
def calculate_org_stats(df):
    '''
    Calculates key statistics for organisations in the GTR including:
    
        Total number of projects
        Total number of projects led
        Total level of funding
        Total level of funding in projects led
        Discipline distribution
        Funder distribution
        Output distribution (projects involving organisation)
        How many collaborations locally
        Scottish local collaborations
        Scottish local collaborations
        Top Scottish collaborator (?)
    
    '''
    
    #Storage for results we will concatenate at the end
    grouped_results = []
    

    #Created a grouped df
    df_by_org = df.groupby('id')
    
    
    #Get the list of projects and collaborations
    proj_rel_lists = pd.concat([df_by_org[var].apply(lambda x: list(x)) for var in ['project_id','rel']],axis=1)
    
    #Number of projects
    proj_rel_lists['project_n'] = [len(x) for x in proj_rel_lists['project_id']]

    #Number of led projects
    proj_rel_lists['lead_project_n'] = [np.sum([x=='LEAD_ORG' for x in statuses]) for statuses in proj_rel_lists['rel']]
    
    #Append to the store
    grouped_results.append(proj_rel_lists)
    
    #Amount of funding
    funding_total = df_by_org['amount'].sum()
    
    #Amount of funding in led projects
    #We subset the org links to only focus on funded projects
    funding_led = df.loc[df['rel']=='LEAD_ORG'].groupby('id')['amount'].sum()
    
    #All funding
    all_funding = pd.concat([funding_total,funding_led],axis=1)
    all_funding.columns = ['all_funding','led_funding']
    
    
    grouped_results.append(all_funding)
    
    #Various distributions of categorical variables
    disc_mix,fund_mix,grant_mix = [df_by_org[var].value_counts().rename(columns={var:'n'} #This is to avoid the conflict when resetting the index
                                                          ).reset_index(drop=False).pivot(
        index='id',columns=var,values=0).fillna(0) for var in ['disc_top','funder','grant_category']]
    
    #Clean column names for grant types and funders (this could be a list comprehension)
    grant_mix.columns = ['grantcat_'+re.sub(' ','_',x.lower()) for x in grant_mix.columns]
    fund_mix.columns = ['funder_'+re.sub(' ','_',x.lower()) for x in fund_mix.columns]
    
    
    grouped_results.append(pd.concat([disc_mix,fund_mix,grant_mix],axis=1))
    
    #Outputs
    #Get the variables
    output_vars = [x for x in df.columns if 'out_' in x]
    
    #Calculate sum  of outputs over all papers
    output_mix = df_by_org[output_vars].sum()
    
    grouped_results.append(output_mix)
    
    #Local collaborators:
    #Create a local analysis df
    collabs = pd.concat([df_by_org[var].sum() for var in ['local_collab','scot_collab']],axis=1)
    
    #Cast as int
    collabs['local_collab'],collabs['scot_collab']= [collabs[v].astype(int) for v in ['local_collab','scot_collab']]
    
    #print(collabs['local_collab'].head())
    
    
    grouped_results.append(collabs)
    
    #return(grouped_results)
    
    return(pd.concat(grouped_results,axis=1))
    
    
    

In [ ]:
org_df = calculate_org_stats(org_proj)

In [ ]:
#And now we need to add names and LADs

org_df['name'],org_df['lad'] = [[df[org_id][var] if org_id in df.keys() else np.nan for org_id in org_df.index] for df,var in zip(
    [org_name_lookup,org_lad_lookup],['name','lad_name'])]



In [ ]:
org_df.to_csv(f'../data/processed/{today_str}_organisation_activities.csv',compression='zip')